<a href="https://colab.research.google.com/github/Rodrigoveloso7/dimensionamento-desumidificador/blob/main/Dimensionamento_Desumidificador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [154]:
import numpy as np


In [155]:
# pip install CoolProp

In [156]:
from CoolProp.CoolProp import PropsSI


#Condições de operação Biogás

In [157]:
# pip install ht

In [158]:
import ht as ht
import ht.boiling_flow as bf

In [159]:
fracoes_volumetricas = {"Methane":0.637,"CarbonDioxide":0.35,"Water":0.013}

In [160]:
P_entrada_biogas = 33.34*1000 # Pa
T_entrada_biogas = 273.15+65 # K
T_referencia = 273.15 # K
P_referencia = 101325 # Pa
vazao_normal_biogas = 60 #Nm³/h
T_alvo_biogas = 273.15 # K
T_alvo_arrefecimento = 273.15-2 # K
T_ambiente = 273.15+25 # K
P_ambiente = 101325 # Pa

#Propriedades geométricas

In [161]:
diam_casco = 330/1000 #metros
diam_hel_serp = 270/1000 # m
diam_entrada_arrefecimento = 25.4/1000
dext_serp = 12.7/1000 # m
espessura_serp = 1.2/1000 # m
diam_interno_serp = dext_serp-2*espessura_serp # m
comprimento_casco = 1995/1000 # metros
volume_arrefecimento = ((np.pi*((diam_casco)**2))/4)*comprimento_casco




#Condições de operação líquido de arrefecimento

https://www.engineeringtoolbox.com/ethylene-glycol-d_146.html

Considerando uma mistura 80% água e 20% monoetilenoglicol para obter uma temperatura de solidificação de -7.9C

In [162]:
cp_arrefecimento = 3.951*1000 # J/kg.K
densidade_arrefecimento = 1030 # kg/m³
viscosidade_dinamica_arrefecimento = 1.42/1000 # Pa.s Este valor foi obtido em cima de fracao massica
k_arrefecimento = 0.55 # W/mK
massa_arrefecimento = volume_arrefecimento*densidade_arrefecimento #kg
massa_arrefecimento
tempo_arrefecimento = 3600 # segundos

In [163]:
vazao_volumetrica_bomba = 1.8/3600 #m³/s
vazao_massica_bomba = vazao_volumetrica_bomba*densidade_arrefecimento
u_casco = vazao_volumetrica_bomba/((np.pi*((diam_entrada_arrefecimento))**(2))/4)
u_casco = u_casco*0.5

In [164]:
vazao_volumetrica_biogas_entrada = (vazao_normal_biogas*(P_referencia/P_entrada_biogas)*(T_entrada_biogas/T_referencia))/3600 #m³/s
densidade_operacao_biogas=0
massa_molar_total_entrada = 0
for elementos in fracoes_volumetricas:
    massa_molar_total_entrada = massa_molar_total_entrada + fracoes_volumetricas[elementos]*PropsSI('M', 'T', T_entrada_biogas, 'P', P_entrada_biogas, elementos)
densidade_operacao_biogas = (P_entrada_biogas*massa_molar_total_entrada)/(8.314*T_entrada_biogas)
vazao_massica_biogas = vazao_volumetrica_biogas_entrada*densidade_operacao_biogas


In [165]:
T_sat = 273.15-10
fluid = 'R22'
divisoes = 100
superaquecimento = 8
T_range_superaquecimento = np.linspace(T_sat,T_sat+ superaquecimento,divisoes)
k_inox = 16
Re_casco = (densidade_arrefecimento*u_casco*diam_hel_serp)/(viscosidade_dinamica_arrefecimento)
Pr_casco = (cp_arrefecimento*viscosidade_dinamica_arrefecimento)/(k_arrefecimento)

In [166]:
def cp_biogas(P_biogas,T_biogas):
   massa_molar_total = 0
   cp_biogas = 0
   for elementos in fracoes_volumetricas:
    massa_molar_total = massa_molar_total + fracoes_volumetricas[elementos]*PropsSI('M', 'T', T_biogas, 'P', P_biogas, elementos)
   #print(massa_molar_total)
   for elementos in fracoes_volumetricas:
    cp_biogas = cp_biogas + (((PropsSI('M', 'T', T_biogas, 'P', P_biogas*fracoes_volumetricas[elementos], elementos)*fracoes_volumetricas[elementos])/massa_molar_total)*PropsSI('C', 'T', T_biogas, 'P', P_biogas*fracoes_volumetricas[elementos], elementos))
   return cp_biogas

In [167]:
cp_biogas(33.34*1000,338.15)

1465.4755020967925

In [168]:
def carga_termica():
  divisoes = 200
  Qsens_biogas = 0
  T = np.linspace(T_entrada_biogas,T_alvo_biogas,divisoes)
  for i in range(len(T)-1):
    cp = cp_biogas(P_entrada_biogas,(T[i]+T[i+1])/2)
    Qsens_biogas = vazao_massica_biogas*cp*(T[i] - T[i+1]) + Qsens_biogas
    cp= 0
  Qlat_biogas = vazao_massica_biogas*(fracoes_volumetricas['Water']*PropsSI('M', 'T', T_entrada_biogas, 'P', P_entrada_biogas, elementos)/massa_molar_total_entrada)*(((PropsSI("H", "T", T_alvo_biogas, "Q", 1, "Water")) - PropsSI("H", "T", T_alvo_biogas, "Q", 0, "Water")))
  Q_arrefecimento = massa_arrefecimento*cp_arrefecimento*(T_ambiente - T_alvo_arrefecimento)/tempo_arrefecimento
  Qtotal = Qlat_biogas + Qsens_biogas +Q_arrefecimento
  return Qtotal,Qlat_biogas,Qsens_biogas,Q_arrefecimento

In [169]:
carga_termica()

(np.float64(7409.326785945957),
 435.55320814719005,
 np.float64(1765.8310980969145),
 5207.942479701853)

#Calculando comprimento evaporador

Definição da vazão mássica a partir da carga térmica, utilizando software CoolSelector da Danfoss


In [170]:
vazao_massica_refri = 169.8/3600 #kg/s
T_evap = 273.15-10 # K
P_entrada_evap = 450*1000 #Pa
x_ini = 0.3 # fração de vapor na entrada do evaporador
Q_lat_refr = vazao_massica_refri*(((PropsSI("H", "T", T_evap, "Q", 1, "R22")) - PropsSI("H", "T", T_evap, "Q", 0.3, "R22")))
Q_lat_refr # W

7025.719327868491

https://www.sciencedirect.com/science/article/abs/pii/S0140700714002369

https://jingweizhu.weebly.com/uploads/1/3/5/4/13548262/summary_of_chen%E2%80%99s_correlation_for_boiling_heat_transfer_to_saturated_fluids_in_convective_flow.pdf

Esse e descrição do chat gpt


In [171]:
# Propriedades obtidas na linha de saturação
rhol = PropsSI('D', 'T', T_evap, 'Q', 0, fluid)       # kg/m³
rhog = PropsSI('D', 'T', T_evap, 'Q', 1, fluid)       # kg/m³
mul  = PropsSI('V', 'T', T_evap, 'Q', 0, fluid)       # Pa.s
mug  = PropsSI('V', 'T', T_evap, 'Q', 1, fluid)       # Pa.s
kl   = PropsSI('L', 'T', T_evap, 'Q', 0, fluid)       # W/m.K
Cpl  = PropsSI('C', 'T', T_evap, 'Q', 0, fluid)       # J/kg.K
Hvap = PropsSI('H', 'T', T_evap, 'Q', 1, fluid) - \
      PropsSI('H', 'T', T_evap, 'Q', 0, fluid)       # J/kg
sigma = PropsSI('SURFACE_TENSION', 'T', T_evap, 'Q', 0, fluid)  # N/m


x = np.linspace(x_ini,0.99, divisoes)
x

array([0.3       , 0.3069697 , 0.31393939, 0.32090909, 0.32787879,
       0.33484848, 0.34181818, 0.34878788, 0.35575758, 0.36272727,
       0.36969697, 0.37666667, 0.38363636, 0.39060606, 0.39757576,
       0.40454545, 0.41151515, 0.41848485, 0.42545455, 0.43242424,
       0.43939394, 0.44636364, 0.45333333, 0.46030303, 0.46727273,
       0.47424242, 0.48121212, 0.48818182, 0.49515152, 0.50212121,
       0.50909091, 0.51606061, 0.5230303 , 0.53      , 0.5369697 ,
       0.54393939, 0.55090909, 0.55787879, 0.56484848, 0.57181818,
       0.57878788, 0.58575758, 0.59272727, 0.59969697, 0.60666667,
       0.61363636, 0.62060606, 0.62757576, 0.63454545, 0.64151515,
       0.64848485, 0.65545455, 0.66242424, 0.66939394, 0.67636364,
       0.68333333, 0.69030303, 0.69727273, 0.70424242, 0.71121212,
       0.71818182, 0.72515152, 0.73212121, 0.73909091, 0.74606061,
       0.7530303 , 0.76      , 0.7669697 , 0.77393939, 0.78090909,
       0.78787879, 0.79484848, 0.80181818, 0.80878788, 0.81575

CORRELAÇÃO CHEN

In [172]:
def correlacao_chen(x):
  # Fluido e condições de operação
  fluid = 'R22'
  T_sat = 273.15-10  # Temperatura de saturação em K (100 °C)
  Te = 8        # Superaquecimento da parede (T_wall - T_sat) em K         # Título de vapor
  # Derivada da pressão de saturação em função da temperatura (dPsat/dT)
  delta_T = 0.01  # K
  p1 = PropsSI('P', 'T', T_sat - delta_T, 'Q', 0, fluid)
  p2 = PropsSI('P', 'T', T_sat + delta_T, 'Q', 0, fluid)
  dPsat = (p2 - p1) / (2 * delta_T)  # Pa/K

  # Cálculo do coeficiente de convecção
  h = bf.Chen_Bennett(vazao_massica_refri, x, diam_interno_serp, rhol, rhog, mul, mug, kl, Cpl, Hvap, sigma, dPsat, Te)
  return h


CORRELAÇÃO CHURCHILL BERNSTEIN

In [173]:
def correlacao_churchill_bernstein(Re, Pr):
  nu_ext = ht.conv_external.Nu_cylinder_Churchill_Bernstein(Re, Pr)
  h_externo = nu_ext*k_arrefecimento/(diam_hel_serp)
  return h_externo

CORRELAÇÃO DITTUS BOELTER

In [174]:
def correlacao_dittus_boelter(Re, Pr):

  h = ht.conv_internal.turbulent_Dittus_Boelter(Re, Pr, heating=False, revised=True)

  return h


In [175]:
L_evap_vapor = 0
for i in range(len(T_range_superaquecimento)-1):
  Q_sup = vazao_massica_refri*PropsSI('C', 'T', (T_range_superaquecimento[i]+T_range_superaquecimento[i+1])/2, 'P', P_entrada_evap, 'R22')*(T_range_superaquecimento[i+1]-T_range_superaquecimento[i])
  velocidade_refrigerante_vapor = vazao_massica_refri/((np.pi*(diam_interno_serp**2)/4)*(PropsSI('D', 'T', T_range_superaquecimento[i]+T_range_superaquecimento[i+1]/2, 'P', P_entrada_evap, 'R22')))
  reynolds_vapor_refr = (velocidade_refrigerante_vapor*diam_interno_serp*PropsSI('D', 'T', (T_range_superaquecimento[i]+T_range_superaquecimento[i+1])/2, 'P', P_entrada_evap, 'R22'))/(PropsSI('V', 'T', T_range_superaquecimento[i]+T_range_superaquecimento[i+1]/2, 'P', P_entrada_evap, 'R22'))
  Pr_vapor = (PropsSI('C', 'T', (T_range_superaquecimento[i]+T_range_superaquecimento[i+1])/2, 'P', P_entrada_evap, 'R22')*PropsSI('V', 'T', (T_range_superaquecimento[i]+T_range_superaquecimento[i+1])/2, 'P', P_entrada_evap, 'R22'))/ (PropsSI('L', 'T', (T_range_superaquecimento[i]+T_range_superaquecimento[i+1])/2, 'P', P_entrada_evap, 'R22'))
  L_evap_vapor_temp = Q_sup*(((np.log(dext_serp/diam_interno_serp))/(2*np.pi*k_inox))+(1/correlacao_dittus_boelter(reynolds_vapor_refr, Pr_vapor)*np.pi*diam_interno_serp))+(1/(correlacao_churchill_bernstein(Re_casco, Pr_casco)*np.pi*dext_serp))*(1/(T_alvo_arrefecimento-T_evap))
  L_evap_vapor = L_evap_vapor + L_evap_vapor_temp
L_evap_vapor

np.float64(1.1323072537169514)

In [176]:
L_evap = 0
for i in range(len(x)-1):
  Q_lat_refr = vazao_massica_refri*(((PropsSI("H", "T", T_evap, "Q", x[i+1], "R22")) - PropsSI("H", "T", T_evap, "Q", x[i], "R22")))
  L_evap_temp = Q_lat_refr*(((np.log(dext_serp/diam_interno_serp))/(2*np.pi*k_inox))+(1/(correlacao_chen(x[i])*np.pi*diam_interno_serp))+(1/(correlacao_churchill_bernstein(Re_casco, Pr_casco)*np.pi*dext_serp)))*(1/(T_alvo_arrefecimento-T_evap))
  L_evap = L_evap + L_evap_temp
L_evap

np.float64(23.605283576047597)

In [177]:
L_serpentina = L_evap+L_evap_vapor
L_serpentina

np.float64(24.73759082976455)

Cálculo região de superaquecimento

#Dimensionamento tubos colméia

Propriedades geométricas

In [178]:
dext_tubos = 19.05/1000 # m
espessura_tubos = 1.2/1000 # m
dint_tubos = dext_tubos-2*espessura_tubos # m
A_int_tubos = np.pi*(dint_tubos**2)/4 # m²
A_ext_tubos = np.pi*(dext_tubos**2)/4 # m²
N = 66

In [179]:
def visc_dinamica_biogas(P_biogas,T_biogas):
   massa_molar_total = 0
   visc_dinamica_biogas = 0
   for elementos in fracoes_volumetricas:
    massa_molar_total = massa_molar_total + fracoes_volumetricas[elementos]*PropsSI('M', 'T', T_biogas, 'P', P_biogas, elementos)
   #print(massa_molar_total)
   for elementos in fracoes_volumetricas:
    visc_dinamica_biogas = visc_dinamica_biogas + (((PropsSI('M', 'T', T_biogas, 'P', P_biogas*fracoes_volumetricas[elementos], elementos)*fracoes_volumetricas[elementos])/massa_molar_total)*PropsSI('V', 'T', T_biogas, 'P', P_biogas*fracoes_volumetricas[elementos], elementos))
   return visc_dinamica_biogas

In [180]:
def condutividade_biogas(P_biogas,T_biogas):
   massa_molar_total = 0
   condutividade_biogas = 0
   for elementos in fracoes_volumetricas:
    massa_molar_total = massa_molar_total + fracoes_volumetricas[elementos]*PropsSI('M', 'T', T_biogas, 'P', P_biogas, elementos)
   #print(massa_molar_total)
   for elementos in fracoes_volumetricas:
    condutividade_biogas = condutividade_biogas + (((PropsSI('M', 'T', T_biogas, 'P', P_biogas*fracoes_volumetricas[elementos], elementos)*fracoes_volumetricas[elementos])/massa_molar_total)*PropsSI('L', 'T', T_biogas, 'P', P_biogas*fracoes_volumetricas[elementos], elementos))
   return condutividade_biogas

In [181]:
def densidade_biogas(P_biogas,T_biogas):
   massa_molar_total = 0
   densidade_biogas = 0
   for elementos in fracoes_volumetricas:
    massa_molar_total = massa_molar_total + fracoes_volumetricas[elementos]*PropsSI('M', 'T', T_biogas, 'P', P_biogas, elementos)
   #print(massa_molar_total)
   densidade_biogas = (P_biogas*massa_molar_total)/(8.314*T_biogas)
   return densidade_biogas

In [182]:
P_entrada_biogas
T_entrada_biogas

338.15

In [183]:
condutividade_biogas(P_entrada_biogas,T_entrada_biogas)

0.027706641514005297

In [184]:
visc_dinamica_biogas(P_entrada_biogas,T_entrada_biogas)

1.5018094295653462e-05

In [185]:
densidade_biogas(P_entrada_biogas,T_entrada_biogas)


0.3066355519540428

In [186]:
Rt_cond_tubos = np.log(dext_tubos/dint_tubos)/(2*np.pi*k_inox)
vazao_massica_tubos = (vazao_massica_biogas/N)
print(vazao_massica_biogas)
print(vazao_massica_tubos)
Q=0
L_tubos_sen = 0
divisoes = 100
T = np.linspace(T_entrada_biogas,T_alvo_biogas,divisoes)
print(P_entrada_biogas)
for i in range(len(T)-1):
  vel_tubos = (vazao_massica_tubos/(((densidade_biogas(P_entrada_biogas,(T[i+1]+T[i])/2)))*A_int_tubos))
  # print(vel_tubos)
  reynolds_biogas_sens = ((vel_tubos*dint_tubos*densidade_biogas(P_entrada_biogas,(T[i+1]+T[i])/2)))/(visc_dinamica_biogas(P_entrada_biogas,(T[i+1]+T[i])/2))
  # print(reynolds_biogas_sens)
  Pr_biogas_sens = (cp_biogas(P_entrada_biogas,(T[i+1]+T[i])/2) *visc_dinamica_biogas(P_entrada_biogas,(T[i+1]+T[i])/2) )/(condutividade_biogas(P_entrada_biogas,(T[i+1]+T[i])/2))

  nud_db_biogas_sens = 0.023*(reynolds_biogas_sens**(0.8))*(Pr_biogas_sens**(0.4))
  nud_db_biogas_sens
  h_conv_tubos_sens = (nud_db_biogas_sens*condutividade_biogas(P_entrada_biogas,(T[i+1]+T[i])/2)/(dint_tubos))*1.3
  fator_atrito = (64/reynolds_biogas_sens)

  # print(h_conv_tubos_sens)
  # print(h_conv_tubos_sens)
  Rt_conv_tubos = (1/(h_conv_tubos_sens*np.pi*dint_tubos))
  Q_bio_sens = vazao_massica_tubos*cp_biogas(P_entrada_biogas,(T[i+1]+T[i])/2)*(-T[i+1]+T[i])
  L = Q_bio_sens*(Rt_cond_tubos+Rt_conv_tubos)/((((T[i+1]+T[i])/2)-T_alvo_arrefecimento))
  Q = Q_bio_sens + Q
  L_tubos_sen = L_tubos_sen + L
  queda_pressao = fator_atrito*L*densidade_biogas(P_entrada_biogas,(T[i+1]+T[i])/2)*(vel_tubos**2)/(2*dint_tubos)
  P_entrada_biogas = P_entrada_biogas - queda_pressao
  # print(queda_pressao)
  # print(Q
  L =0
  # print(Q)
print(L_tubos_sen)
print(P_entrada_biogas)
print(Q)

0.019227838241768682
0.00029133088245104064
33340.0
1.8930669477289555
33329.331774079605
26.755013338286304
